In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate

from plugin.managers.data_manager import DataManager
from spectrumlab.peaks.analyte_peaks.intensity.transformers import (
    RegressionIntensityTransformer,
    estimate_bounds,
    process_frame,
)


In [ ]:
data_manager = DataManager()
atom_data = data_manager.parse()

column_id, *_ = atom_data.data.keys()
frame = process_frame(atom_data.data[column_id].frame)
bounds = atom_data.data[column_id].bounds
nickname = atom_data.data[column_id].nickname

In [ ]:
transformer = RegressionIntensityTransformer.create(
    data=frame,
    bounds=estimate_bounds(frame, threshold=.05),
)
transformer.show()

In [ ]:
frame['intensity_hat'] = transformer.apply(frame['intensity'])
frame[['intensity', 'intensity_hat']].head(50)

In [ ]:
%matplotlib inline

fig, (ax_left, ax_right) = plt.subplots(ncols=2, figsize=(12, 4))

plt.title(nickname)

#
plt.sca(ax_left)

x = np.linspace(0, 20, 501)
y = transformer.apply(x)
plt.plot(
    x, y,
    color='red', linestyle='-',
    label=r'$\hat{R}$',
)

plt.plot(
    frame['intensity'], frame['intensity_hat'],
    color='black', linestyle='--',
    label=r'$\gamma$ (approx)',
)

plt.xlabel(r'$R$')
plt.ylabel(r'$\hat{R}$')
plt.grid(color='grey', linestyle=':')
plt.legend()

#
plt.sca(ax_right)

x = np.linspace(0, 20, 501)
y = transformer.apply(x)
y_hat = interpolate.interp1d(
    frame['intensity'], frame['intensity_hat'],
    kind='linear',
    bounds_error=False,
    fill_value=np.nan,
)(x)
plt.plot(
    x, 100*(y_hat - y)/y,
    color='black', linestyle='-',
    label='true',
)


plt.xlabel(r'$R$')
plt.ylabel(r'$(\gamma - \hat{R})/\hat{R}, \%$')
plt.grid(color='grey', linestyle=':')
plt.legend()


plt.show()